In [348]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json


In [350]:
with open('healthcare-claims-prediction_kaggel/patient_data_train.json', 'r') as file:
    train_data_json = json.load(file)
with open('healthcare-claims-prediction_kaggel/patient_data_test.json', 'r') as file:
    test_data_json = json.load(file)

train_claims = pd.read_csv('healthcare-claims-prediction_kaggel/train.csv')

def normalize_and_clean_data(data):
    '''
    Normalizing and handling the data
    '''
    import warnings
    warnings.filterwarnings("ignore")
    df = pd.json_normalize(data)

    # each patient is unique record,i will be replacing nulls with 0 values

    # so First we will do this for the features related to past years

    df["Conditions.HD"].fillna(0,inplace=True)

    df["Conditions.HT"].fillna(0,inplace=True)

    df["Conditions.DB"].fillna(0,inplace=True)

    df["Conditions.AT"].fillna(0,inplace=True)

    return df

train_data = normalize_and_clean_data(train_data_json)
test_data = normalize_and_clean_data(test_data_json)
    # As we already know, we have only 5 past years to handle the values
    # Now, we need to do the similar kind of analysis for conditions features as well.



train_data = pd.merge(train_data, train_claims, on='PatientID')


train_data.isnull().sum()

PatientID                    0
Sex                          0
Age                          0
Conditions.HD                0
Conditions.HT                0
Out patient costs.2023     748
Out patient costs.2022    1538
Out patient costs.2021    2278
Out patient costs.2020    3060
Out patient costs.2019    3857
Conditions.DB                0
Conditions.AT                0
TotalClaims                  0
dtype: int64

In [360]:
!pip install category_encoders


   ---------------------------------------- 0.0/82.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/82.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/82.0 kB ? eta -:--:--
   ------------------- -------------------- 41.0/82.0 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 82.0/82.0 kB 763.4 kB/s eta 0:00:00


In [370]:
import pandas as pd
import numpy as np
#import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from category_encoders import TargetEncoder
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
import optuna

# Load train and test data
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Helper function to fill missing values using year-to-year differences
def fill_na_respectively(row, x, mean_val, feature_1, feature_2):
    if pd.isna(row[feature_2]):
        if pd.isna(row[feature_1]):
            row[feature_2] = mean_val
        else:
            row[feature_2] = row[feature_1] - x
    return row

# Calculate differences for each year-to-year pair and fill missing values accordingly
respective_dif = {}
def find_dif(df, feature_1, feature_2):
    df_filtered = df.dropna(subset=[feature_1, feature_2])
    name = feature_1 + feature_2
    df_filtered["difference"] = df_filtered[feature_1] - df_filtered[feature_2]
    respective_dif[name] = df_filtered["difference"].mean()

find_dif(train_df, "Out patient costs.2023", "Out patient costs.2022")
find_dif(train_df, "Out patient costs.2022", "Out patient costs.2021")
find_dif(train_df, "Out patient costs.2021", "Out patient costs.2020")
find_dif(train_df, "Out patient costs.2020", "Out patient costs.2019")

def fill_yearly_costs(df):
    mean_values = df["Out patient costs.2023"].mean()
    df["Out patient costs.2023"] = df["Out patient costs.2023"].fillna(mean_values)
    for year1, year2 in [("2023", "2022"), ("2022", "2021"), ("2021", "2020"), ("2020", "2019")]:
        feature_1 = f"Out patient costs.{year1}"
        feature_2 = f"Out patient costs.{year2}"
        name = feature_1 + feature_2
        mean_val = df[feature_2].mean()
        df = df.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)
    return df

train_df = fill_yearly_costs(train_df)
test_df = fill_yearly_costs(test_df)

# Advanced Imputation: Combining KNN and Iterative Imputation
knn_imputer = KNNImputer(n_neighbors=5)
train_df_imputed = knn_imputer.fit_transform(train_df.select_dtypes(include=[np.number]))
train_df[train_df.select_dtypes(include=[np.number]).columns] = train_df_imputed

imputer = IterativeImputer(max_iter=10, random_state=42)
train_df_imputed = imputer.fit_transform(train_df.select_dtypes(include=[np.number]))
train_df[train_df.select_dtypes(include=[np.number]).columns] = train_df_imputed

# Cluster-based feature engineering: Adding a cluster label
def add_cluster_feature(df, cols, n_clusters=5):
    km = KMeans(n_clusters=n_clusters, random_state=42)
    df_scaled = StandardScaler().fit_transform(df[cols])
    df["Cluster_Label"] = km.fit_predict(df_scaled)
    return df

train_df = add_cluster_feature(train_df, [f"Out patient costs.{year}" for year in range(2019, 2024)])
test_df = add_cluster_feature(test_df, [f"Out patient costs.{year}" for year in range(2019, 2024)])

# Target Encoding for categorical variable 'Sex' based on 'TotalClaims'
encoder = TargetEncoder()
train_df["Sex"] = encoder.fit_transform(train_df["Sex"], train_df["TotalClaims"])
test_df["Sex"] = encoder.transform(test_df["Sex"])

# Define features and target, removing 'TotalClaims' and 'PatientID' from training features
x = train_df.drop(columns=["TotalClaims", "PatientID"], errors="ignore")
y = train_df["TotalClaims"]
test_data = test_df.drop(columns=["PatientID"], errors="ignore")

# Outlier Treatment: Cap extreme values based on quantiles
for column in x.columns:
    q_low = x[column].quantile(0.01)
    q_high = x[column].quantile(0.99)
    x[column] = np.clip(x[column], q_low, q_high)
# These lines are making both the features in simlar order

temp = test_df[["Conditions.AT","Conditions.DB"]]
test_data.drop(["Conditions.AT","Conditions.DB"], axis=1, inplace=True)
test_data["Conditions.DB"] = temp["Conditions.DB"]
test_data["Conditions.AT"] = temp["Conditions.AT"]


# Ensure consistent column order between x and test_data
test_data = test_data[x.columns]

# Apply Polynomial Feature Expansion and keep column names consistent
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
x_poly = poly.fit_transform(x)
test_data_poly = poly.transform(test_data)

# Convert to DataFrame to ensure consistency in feature names
x_poly = pd.DataFrame(x_poly, columns=poly.get_feature_names_out(input_features=x.columns))
test_data_poly = pd.DataFrame(test_data_poly, columns=poly.get_feature_names_out(input_features=test_data.columns))

# Scaling the data consistently
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_poly)
test_data_scaled = scaler.transform(test_data_poly)

# Convert scaled arrays back to DataFrames for consistency in feature names
x_scaled = pd.DataFrame(x_scaled, columns=x_poly.columns)
test_data_scaled = pd.DataFrame(test_data_scaled, columns=test_data_poly.columns)

# Split data for training and validation
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=42)

# You can now proceed to model training and evaluation with the preprocessed data


In [ ]:
# Optuna objective function for tuning
def objective(trial):
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 5)
    }
    
    catboost_params = {
        'iterations': trial.suggest_int('iterations', 500, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1)
    }
    
    lgbm_params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 5)
    }
    
    # Models with trial parameters
    xgb_tuned = XGBRegressor(**xgb_params, objective='reg:squarederror', random_state=42)
    catboost_tuned = CatBoostRegressor(**catboost_params, random_state=42, verbose=0)
    lgbm_tuned = LGBMRegressor(**lgbm_params, objective='regression', random_state=42)
    
    # Stacking with GradientBoostingRegressor as meta-model
    meta_model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, max_depth=5, random_state=42)
    stacking_model = StackingRegressor(
        estimators=[('xgb', xgb_tuned), ('catboost', catboost_tuned), ('lgbm', lgbm_tuned)],
        final_estimator=meta_model,
        cv=KFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1
    )
    
    # Fit stacking model and evaluate
    stacking_model.fit(x_train, y_train)
    y_pred_test = stacking_model.predict(x_test)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    if test_mae <= 400:
        trial.study.stop()
    
    return test_mae

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# Retrieve best trial
best_trial = study.best_trial
print(f"Best MAE: {best_trial.value}")
print(f"Best params: {best_trial.params}")

[I 2024-10-25 16:29:23,325] A new study created in memory with name: no-name-b434aeff-b06e-4157-bf81-b050d476c473
[I 2024-10-25 16:30:50,076] Trial 0 finished with value: 1273.201218101874 and parameters: {'n_estimators': 918, 'learning_rate': 0.022165310781723475, 'max_depth': 8, 'subsample': 0.8200406732584331, 'colsample_bytree': 0.9538552172655104, 'reg_lambda': 3.6554786669417245, 'reg_alpha': 3.7075742488669787, 'iterations': 851, 'depth': 4, 'l2_leaf_reg': 2.640773208302611, 'bagging_temperature': 0.9067733354545271}. Best is trial 0 with value: 1273.201218101874.
[I 2024-10-25 16:32:19,131] Trial 1 finished with value: 927.103488733384 and parameters: {'n_estimators': 918, 'learning_rate': 0.05896207438377008, 'max_depth': 5, 'subsample': 0.7027435110237576, 'colsample_bytree': 0.9097798340927015, 'reg_lambda': 1.4054382706483584, 'reg_alpha': 0.27377472220700005, 'iterations': 1412, 'depth': 6, 'l2_leaf_reg': 2.457295552737067, 'bagging_temperature': 0.450023924858422}. Best i